# 資料取得

將資料以`send_table`結合`list_table`變成`report_table.csv`
先結合資料，等確定報告需要的欄位順序後再來調整

In [ ]:
import pandas as pd

send_table = pd.read_csv('bquxjob_42ec509c_191abde8232.csv', encoding='utf8')
send_columns = send_table.iloc[1].index.to_list()

list_table = pd.read_csv('bquxjob_72c15597_191abde8993.csv', encoding='utf8')
list_columns = list_table.iloc[1].index.to_list()

In [ ]:
# V1.1
import pandas as pd
from fuzzywuzzy import fuzz
from datetime import datetime, timedelta

def create_monthly_report(df_b, df_c, threshold=6, date_tolerance=0):
    # 轉換日期列
    df_b['senddate'] = pd.to_datetime(df_b['senddate'])
    df_c['senddate'] = pd.to_datetime(df_c['senddate'])
    
    # 創建空的月報表框架
    columns = ['需求單位', 'EDM類型', '寄件標題', '使用標題', '發送日期', '發送檔數', '單檔份數', '主題模型',
               '關鍵字名單編號', '其他篩選條件', 'Launchid', '寄送數量', '收信人數', '收信比例', '開信人數',
               '開信比例', '點擊EDM人數', '點擊比例', '寄送失敗人數', '失敗比例']
    rows = []
    
    # 遍歷寄件表，創建月報表行
    for _, row_b in df_b.iterrows():
        # 在名單表中尋找匹配
        matches_c = df_c[
            (df_c['senddate'] >= row_b['senddate'] - timedelta(days=date_tolerance)) &
            (df_c['senddate'] <= row_b['senddate'] + timedelta(days=date_tolerance))
        ]
        
        if not matches_c.empty:
            matches_c['similarity'] = matches_c['topic'].apply(lambda x: fuzz.ratio(x, row_b['subject']))
            best_match_c = matches_c.loc[matches_c['similarity'].idxmax()]
            
            if best_match_c['similarity'] >= threshold:
                new_row = {
                    '需求單位': '',  # 空白欄位
                    'EDM類型': '',  # 空白欄位
                    '寄件標題': row_b['subject'],
                    '使用標題': best_match_c['topic'],
                    '發送日期': row_b['senddate'],
                    '發送檔數': '',  # 空白欄位
                    '單檔份數': '',  # 空白欄位
                    '主題模型': best_match_c['priority'].split('_')[-1] if '_' in best_match_c['priority'] else best_match_c['priority'],
                    '關鍵字名單編號': '',  # 空白欄位
                    '其他篩選條件': '',  # 空白欄位
                    'Launchid': row_b['launchid'],
                    '寄送數量': row_b['send'],
                    '收信人數': row_b['receive'],
                    '收信比例': row_b['receive_rate'],
                    '開信人數': row_b['open'],
                    '開信比例': row_b['open_rate'],
                    '點擊EDM人數': row_b['click'],
                    '點擊比例': row_b['click_rate'],
                    '寄送失敗人數': row_b['fail'],
                    '失敗比例': row_b['fail_rate']
                }
                rows.append(new_row)
    
    return pd.DataFrame(rows, columns=columns)

# 讀取CSV文件
df_b = pd.read_csv('bquxjob_42ec509c_191abde8232.csv')
df_c = pd.read_csv('bquxjob_72c15597_191abde8993.csv')

# 創建月報表
result = create_monthly_report(df_b, df_c)

# 保存結果
result.to_csv('月報表.csv', index=False, encoding='big5')
print(f"生成的月報表行數: {len(result)}")
print("月報表已創建，結果已保存到 '月報表.csv'")

# 打印一些調試信息
print(f"df_b 的行數: {len(df_b)}")
print(f"df_c 的行數: {len(df_c)}")
print("df_b 的前幾行:")
print(df_b.head())
print("df_c 的前幾行:")
print(df_c.head())

In [ ]:
# V2.0
import pandas as pd
from fuzzywuzzy import fuzz
from datetime import datetime, timedelta

def create_monthly_report(send_table, list_table, threshold=6, date_tolerance=0):
    # 轉換日期列
    send_table['senddate'] = pd.to_datetime(send_table['senddate'])
    list_table['senddate'] = pd.to_datetime(list_table['senddate'])
    
    # 創建空的月報表框架
    columns = ['需求單位', 'EDM類型', '寄件標題', '使用標題', '發送日期', '發送檔數', '單檔份數', '主題模型',
               '關鍵字名單編號', '其他篩選條件', 'Launchid', '寄送數量', '收信人數', '收信比例', '開信人數',
               '開信比例', '點擊EDM人數', '點擊比例', '寄送失敗人數', '失敗比例']
    rows = []
    
    # 遍歷寄件表，創建月報表行
    for _, row_b in send_table.iterrows():
        # 在名單表中尋找匹配
        matches_c = list_table[
            (list_table['senddate'] >= row_b['senddate'] - timedelta(days=date_tolerance)) &
            (list_table['senddate'] <= row_b['senddate'] + timedelta(days=date_tolerance))
        ]
        
        if not matches_c.empty:
            matches_c['similarity'] = matches_c['topic'].apply(lambda x: fuzz.ratio(x, row_b['subject']))
            best_match_c = matches_c.loc[matches_c['similarity'].idxmax()]
            
            if best_match_c['similarity'] >= threshold:
                priority_parts = best_match_c['priority'].split('_')
                main_model = priority_parts[1] if len(priority_parts) > 1 else ''
                keyword_list = priority_parts[2] if len(priority_parts) > 2 else ''
                
                new_row = {
                    '需求單位': '',  # 空白欄位
                    'EDM類型': '',  # 空白欄位
                    '寄件標題': row_b['subject'],
                    '使用標題': best_match_c['topic'],
                    '發送日期': row_b['senddate'],
                    '發送檔數': '',  # 空白欄位
                    '單檔份數': '',  # 空白欄位
                    '主題模型': main_model,
                    '關鍵字名單編號': keyword_list,
                    '其他篩選條件': '',  # 空白欄位
                    'Launchid': row_b['launchid'],
                    '寄送數量': row_b['send'],
                    '收信人數': row_b['receive'],
                    '收信比例': row_b['receive_rate'],
                    '開信人數': row_b['open'],
                    '開信比例': row_b['open_rate'],
                    '點擊EDM人數': row_b['click'],
                    '點擊比例': row_b['click_rate'],
                    '寄送失敗人數': row_b['fail'],
                    '失敗比例': row_b['fail_rate']
                }
                rows.append(new_row)
    
    return pd.DataFrame(rows, columns=columns)

# 讀取CSV文件
send_table = pd.read_csv('bquxjob_42ec509c_191abde8232.csv')
list_table = pd.read_csv('bquxjob_72c15597_191abde8993.csv')

# 創建月報表
report_table = create_monthly_report(send_table, list_table)

# 保存結果
report_table.to_csv('月報表2.csv', index=False, encoding='big5')
print(f"生成的月報表行數: {len(report_table)}")
print("月報表已創建，結果已保存到 '月報表2.csv'")

# 打印一些調試信息
print(f"send_table 的行數: {len(send_table)}")
print(f"list_table 的行數: {len(list_table)}")
print("send_table 的前幾行:")
print(send_table.head())
print("list_table 的前幾行:")
print(list_table.head())

In [ ]:
# V2.1
import pandas as pd
from fuzzywuzzy import fuzz
from datetime import datetime, timedelta

def get_chinese_number(number):
    ranges = [
        (4600, 6050, '五千'),
        (9700, 13000, '一萬'),
        (19400, 23000, '兩萬'),
        (29100, 33000, '三萬'),
        (38800, 44000, '四萬'),
        (48800, 55000, '五萬'),
        (97500, 115000, '十萬'),
        (185000, 234000, '二十萬'),
        (244000, 282500, '二十五萬'),
        (272500, 331000, '三十萬'),
        (485000, float('inf'), '五十萬以上')
    ]
    
    for low, high, description in ranges:
        if low <= number < high:
            return description
    return '人工確認'

def create_monthly_report(send_table, list_table, threshold=6, date_tolerance=0):
    # 轉換日期列
    send_table['senddate'] = pd.to_datetime(send_table['senddate'])
    list_table['senddate'] = pd.to_datetime(list_table['senddate'])
    
    # 創建空的月報表框架
    columns = ['需求單位', 'EDM類型', '寄件標題', '使用標題', '發送日期', '發送檔數', '單檔份數', '主題模型',
               '關鍵字名單編號', '其他篩選條件', 'Launchid', '寄送數量', '收信人數', '收信比例', '開信人數',
               '開信比例', '點擊EDM人數', '點擊比例', '寄送失敗人數', '失敗比例']
    rows = []
    
    # 遍歷寄件表，創建月報表行
    for _, row_b in send_table.iterrows():
        # 在名單表中尋找匹配
        matches_c = list_table[
            (list_table['senddate'] >= row_b['senddate'] - timedelta(days=date_tolerance)) &
            (list_table['senddate'] <= row_b['senddate'] + timedelta(days=date_tolerance))
        ]
        
        if not matches_c.empty:
            matches_c['similarity'] = matches_c['topic'].apply(lambda x: fuzz.ratio(x, row_b['subject']))
            best_match_c = matches_c.loc[matches_c['similarity'].idxmax()]
            
            if best_match_c['similarity'] >= threshold:
                priority_parts = best_match_c['priority'].split('_')
                main_model = priority_parts[1] if len(priority_parts) > 1 else ''
                keyword_list = priority_parts[2] if len(priority_parts) > 2 else ''
                
                new_row = {
                    '需求單位': '',  # 空白欄位
                    'EDM類型': '',  # 空白欄位
                    '寄件標題': row_b['subject'],
                    '使用標題': best_match_c['topic'],
                    '發送日期': row_b['senddate'],
                    '發送檔數': '',  # 空白欄位
                    '單檔份數': get_chinese_number(row_b['send']),  # 新增：根據send決定單檔份數
                    '主題模型': main_model,
                    '關鍵字名單編號': keyword_list,
                    '其他篩選條件': '',  # 空白欄位
                    'Launchid': row_b['launchid'],
                    '寄送數量': row_b['send'],
                    '收信人數': row_b['receive'],
                    '收信比例': row_b['receive_rate'],
                    '開信人數': row_b['open'],
                    '開信比例': row_b['open_rate'],
                    '點擊EDM人數': row_b['click'],
                    '點擊比例': row_b['click_rate'],
                    '寄送失敗人數': row_b['fail'],
                    '失敗比例': row_b['fail_rate']
                }
                rows.append(new_row)
    
    return pd.DataFrame(rows, columns=columns)

# 讀取CSV文件
send_table = pd.read_csv('bquxjob_42ec509c_191abde8232.csv')
list_table = pd.read_csv('bquxjob_72c15597_191abde8993.csv')

# 創建月報表
report_table = create_monthly_report(send_table, list_table)

# 保存結果
report_table.to_csv('月報表3.csv', index=False, encoding='big5')
print(f"生成的月報表行數: {len(report_table)}")
print("月報表已創建，結果已保存到 '月報表.csv'")

# 打印一些調試信息
print(f"send_table 的行數: {len(send_table)}")
print(f"list_table 的行數: {len(list_table)}")
print("send_table 的前幾行:")
print(send_table.head())
print("list_table 的前幾行:")
print(list_table.head())

假想資料

In [ ]:
import pandas as pd

# 假設這是你匯入的 DataFrame
data = {
    '標題': [
        "從零開始打造爆款產品！行銷大師教你如何快速獲取新客戶",
        "中小企業必修課！如何透過內容行銷提升品牌知名度並降低廣告成本",
        "掌握社群媒體最新趨勢！教你如何用小預算創造大效益",
        "行銷漏斗全解析！從引流到成交，教你打造完整的顧客旅程",
        "限時免費！行銷大師線上講座，教你如何提升網站轉換率"
    ],
    '簡稱': ['從零開始', '內容行銷', '社群媒體', '漏斗全解析', '網站轉換率']
}

df = pd.DataFrame(data)

# 使用 iloc[1] 來選取第二列（index:1）的資料
title_at_index_1 = df.iloc[2]['標題']  # 這會選取標題列中第二個資料

# 或者，如果你想要撈取第二列的所有欄位名稱，可以使用這個方法：
columns_at_index_1 = df.iloc[1].index.tolist()

print("標題名稱（index:1）:", title_at_index_1)
print("所有欄位名稱（index:1）:", columns_at_index_1)


In [ ]:
import pandas as pd

# 生成第一個 DataFrame (a)
data_a = {
    'ID': [1, 2, 3, 4, 5],
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eva'],
    'Age': [25, 30, 35, 40, 45]
}
df_a = pd.DataFrame(data_a)

# 生成第二個 DataFrame (b) - 比第一個 DataFrame 大小不同，且欄位不同
data_b = {
    'ID': [3, 4, 5, 6, 7, 8],
    'City': ['Taipei', 'Kaohsiung', 'Tainan', 'Taichung', 'Hsinchu', 'Keelung'],
    'Salary': [50000, 60000, 70000, 80000, 90000, 100000]
}
df_b = pd.DataFrame(data_b)

# 顯示兩個 DataFrame
print("DataFrame A:")
print(df_a)
print("\nDataFrame B:")
print(df_b)


In [ ]:
# Dataframe合併

df_e = pd.concat([df_a,df_b], axis=1)

# 透過選定的欄位JOIN

df_c = pd.merge(df_a, df_b, on='ID', how='outer')
df_d = pd.merge(df_a, df_b, on='ID', how='inner')